### Carga de archivo results.json (del directorio 21-03-21) en la tabla `f1_processed` sin particiones 

In [0]:
%sql
DROP TABLE f1_raw.results

In [0]:
%sql
DROP DATABASE f1_raw

In [0]:
%sql
DROP TABLE f1_processed.results

In [0]:
%sql
DROP DATABASE f1_processed

Debo subir el archivo results.json que lo saque del directorio 2021-03-21

In [0]:
dbutils.fs.mkdirs("/FileStore/raw")

Out[116]: True

Debo subir el archivo results.json que corresponde a este directorio

In [0]:
dbutils.fs.mkdirs("/FileStore/raw/2021-03-21")

Out[117]: True

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_processed
LOCATION "/FileStore/processed"

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_raw
LOCATION "/FileStore/raw"

In [0]:
%sql
DROP TABLE IF EXISTS f1_raw.results;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.results(
resultId INT,
raceId INT,
driverId INT,
constructorId INT,
number INT,grid INT,
position INT,
positionText STRING,
positionOrder INT,
points INT,
laps INT,
time STRING,
milliseconds INT,
fastestLap INT,
rank INT,
fastestLapTime STRING,
fastestLapSpeed FLOAT,
statusId STRING)
USING JSON
OPTIONS(path "/FileStore/raw/results.json")

In [0]:
raw_folder_path = "/FileStore/raw"

In [0]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [0]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
# Ejemplo para la fecha 2021-03-21: esto seria equivalente a la ruta: /mnt/formula1dl/raw/2021-03-21/results.json
results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/results.json")

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("file_date", lit(v_file_date))

In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn("ingestion_date", current_timestamp())
  return output_df

In [0]:
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

In [0]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

In [0]:
results_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.results")

[![p474.png](https://i.postimg.cc/sX78LK3p/p474.png)](https://postimg.cc/0M5cMYMQ)
[![p475.png](https://i.postimg.cc/KjbVhLYC/p475.png)](https://postimg.cc/HcS3QrXt)

### Carga incremental utilizando ahora el directorio 21-03-21 con el archivo results.json en la tabla `f1_processed`  y usando dynamic y static

In [0]:
%sql
DROP TABLE f1_processed.results

In [0]:
%sql
DROP DATABASE f1_processed

[![p476.png](https://i.postimg.cc/ydHnDKzZ/p476.png)](https://postimg.cc/06V74Ljk)

In [0]:
%sql
CREATE DATABASE f1_processed

Para la **segunda carga** y **tercera carga**: Desde aqui vamos a comenzar a ejecutar

In [0]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
raw_folder_path = "/FileStore/raw"

In [0]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [0]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [0]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
# Ejemplo para la fecha 2021-03-21: esto seria equivalente a la ruta: /mnt/formula1dl/raw/2021-03-21/results.json
results_df = spark.read \
.schema(results_schema) \
.json(f"{raw_folder_path}/{v_file_date}/results.json")

In [0]:
results_with_columns_df = results_df.withColumnRenamed("resultId", "result_id") \
                                    .withColumnRenamed("raceId", "race_id") \
                                    .withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("constructorId", "constructor_id") \
                                    .withColumnRenamed("positionText", "position_text") \
                                    .withColumnRenamed("positionOrder", "position_order") \
                                    .withColumnRenamed("fastestLap", "fastest_lap") \
                                    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
                                    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
                                    .withColumn("file_date", lit(v_file_date))

In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn("ingestion_date", current_timestamp())
  return output_df

In [0]:
results_with_ingestion_date_df = add_ingestion_date(results_with_columns_df)

In [0]:
results_final_df = results_with_ingestion_date_df.drop(col("statusId"))

In [0]:
# Lo usaremos en la segunda carga
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [ ]:
# Lo usaremos en la tercera carga
spark.conf.set("spark.sql.sources.partitionOverwriteMode","static")

In [0]:
results_final_df = results_final_df.select("result_id","driver_id","constructor_id","number","grid","position","position_text","position_order","points","laps","time","milliseconds","fastest_lap","rank","fastest_lap_time","fastest_lap_speed","file_date","ingestion_date", "race_id")

In [0]:
if (spark._jsparkSession.catalog().tableExists("f1_processed.results")):
  results_final_df.write.mode('overwrite').insertInto("f1_processed.results")
else:
  results_final_df.write.partitionBy('race_id').format('parquet').saveAsTable("f1_processed.results")

In [0]:
%sql
SELECT COUNT(DISTINCT(race_id)) FROM f1_processed.results


count(DISTINCT race_id)
1035


Agregue un nuevo registro con el nuevo race_id=1048 al archivo **results.json** del directorio **2021-03-21**, vamos a ver si dynamic agrega ese nuevo registro. Vuelvo a correr el notebook, no debo eliminar nada, solo ejecutarlo.

[![p477.png](https://i.postimg.cc/SRWnJ0kJ/p477.png)](https://postimg.cc/sGgjNNBr)

Y efectivamente, dynamic si agrega el registro. Todo lo demas lo deja como estaba.

In [0]:
%sql
SELECT COUNT(DISTINCT(race_id)) FROM f1_processed.results

count(DISTINCT race_id)
1036


Probemos que sucede con **static**. Voy a agregar un nuevo registro con el nuevo **race_id=1049** al archivo **results.json** del directorio **2021-03-21**, vamos a ver si static agrega ese nuevo registro. Vuelvo a correr el notebook, no debo eliminar nada, solo ejecutarlo.

[![p478.png](https://i.postimg.cc/Dw1xn09T/p478.png)](https://postimg.cc/G4hFjL37)

Tambien me escribio el registro nuevo

In [0]:
%sql
SELECT COUNT(DISTINCT(race_id)) FROM f1_processed.results

count(DISTINCT race_id)
1037


En resumen, dynamic solo SE ENFOCA en la particion modificada y static elimina todo y vuuelve a escribir todo otra vez

[![p479.png](https://i.postimg.cc/6qRMqQ8n/p479.png)](https://postimg.cc/N2s6PB7j)